In [1]:
import time
import random
import numpy as np
import importlib

import sys
sys.path.append("../../")

In [2]:
from qiskit import QuantumProgram
import Qconfig

In [3]:
import tools.Experiment_tools as exptool

In [114]:
importlib.reload(exptool)

<module 'tools.Experiment_tools' from '/home/christophe/Work/Programming/IBMQE/qiskit-sdk-py/examples/experimenting_with_fault_tolerance/tools/Experiment_tools.py'>

In [5]:
qprogram = QuantumProgram()
qprogram.set_api(Qconfig.APItoken, Qconfig.config["url"])

In [6]:
qprogram.available_backends()

['ibmqx4',
 'ibmqx2',
 'ibmqx5',
 'ibmqx_qasm_simulator',
 'local_qasm_simulator',
 'local_unitary_simulator']

In [41]:
qprogram.get_backend_status("ibmqx4")

{'available': True, 'backend': 'ibmqx4', 'busy': False, 'pending_jobs': 35}

In [8]:
config = qprogram.get_backend_configuration("ibmqx4",list_format=True)
config

{'basis_gates': 'SU2+CNOT',
 'chip_name': 'Raven',
 'coupling_map': [[1, 0], [2, 0], [2, 1], [2, 4], [3, 2], [3, 4]],
 'description': '5 qubit transmon bowtie chip 3',
 'n_qubits': 5,
 'name': 'ibmqx4',
 'online_date': '2017-09-18T11:00:00.000Z',
 'simulator': False,
 'url': 'https://ibm.biz/qiskit-ibmqx4',
 'version': '1'}

In [9]:
n = config['n_qubits']
q = qprogram.create_quantum_register("q",n)
c = qprogram.create_classical_register("c",n)

In [10]:
circuit_names = exptool.all_circuits(qprogram,config['coupling_map'])

In [39]:
rand_perm_names = np.random.permutation(circuit_names)
batches = [list(rand_perm_names[50*j:50*(j+1)]) for j in range(0,1+int(len(circuit_names)/50))]

compiled_qobj_list = [qprogram.compile(batch, 
                                       backend='ibmqx4', 
                                       config=None,
                                       silent=False, 
                                       basis_gates=None, 
                                       coupling_map=None, 
                                       initial_layout=None, 
                                       shots=8192, 
                                       max_credits=5, 
                                       seed=None) for batch in batches]

In [40]:
qprogram.run_batch_async(compiled_qobj_list, wait=10, timeout=30, silent=True, callback=exptool.post_treatment_list)
#qprogram.run_batch_async([compiled_minibatch], wait=10, timeout=30, silent=False, callback=exptool.post_treatment_list)

In [12]:
api = qprogram.get_api()

In [179]:
print(exptool.fetch_previous('timed_out.txt',api))
print(exptool.fetch_previous('completed.txt',api))

5
0


In [90]:
with open('data/API_dumps/api_dump_49addd79a37c61fc08b59fe293555dc0.txt','r') as dump_file:
    file_string = dump_file.read()
    job_result = eval(file_string)
print(job_result.keys())
print(job_result['qasms'][0].keys())
print(job_result['calibration'].keys())
[elem['readoutError'] for elem in job_result['calibration']['qubits']]

dict_keys(['qasms', 'shots', 'backend', 'status', 'maxCredits', 'usedCredits', 'creationDate', 'deleted', 'id', 'userId', 'calibration'])
dict_keys(['qasm', 'status', 'executionId', 'data'])
dict_keys(['lastUpdateDate', 'fridgeParameters', 'qubits', 'multiQubitGates', 'version'])


[{'date': '2017-10-11T11:08:37Z', 'value': 0.037},
 {'date': '2017-10-11T11:08:37Z', 'value': 0.055},
 {'date': '2017-10-11T11:08:37Z', 'value': 0.025},
 {'date': '2017-10-11T11:08:37Z', 'value': 0.059},
 {'date': '2017-10-11T11:08:37Z', 'value': 0.039}]

In [94]:
for k in job_result['qasms'][22]['data']['counts']:
    print((k,job_result['qasms'][22]['data']['counts'][k]))
print(len(job_result['qasms'][0]['qasm'].split('\n')))
job_result['qasms'][0]['qasm'].split('\n')

('10000', 7)
('10001', 56)
('10011', 1)
('10100', 103)
('10101', 5)
('10110', 1)
('11001', 3)
('11100', 1)
('00000', 393)
('00001', 3466)
('00010', 7)
('00011', 48)
('00100', 3574)
('00101', 145)
('00110', 40)
('00111', 3)
('01000', 12)
('01001', 157)
('01011', 2)
('01100', 161)
('01101', 5)
('01110', 2)
28


['',
 'include "qelib1.inc";',
 'qreg q[5];',
 'creg c[5];',
 'u2(0.0,3.141592653589793) q[3];',
 'u2(0.0,3.141592653589793) q[2];',
 'cx q[2],q[0];',
 'cx q[2],q[1];',
 'u2(0.0,3.141592653589793) q[2];',
 'cx q[3],q[2];',
 'u2(0.0,3.141592653589793) q[2];',
 'cx q[2],q[4];',
 'u2(0.0,3.141592653589793) q[4];',
 'u1(1.5707963267948966) q[4];',
 'measure q[4] -> c[4];',
 'u2(0.0,3.141592653589793) q[3];',
 'u2(0.0,3.141592653589793) q[3];',
 'u1(1.5707963267948966) q[3];',
 'measure q[3] -> c[3];',
 'u2(0.0,3.141592653589793) q[1];',
 'u1(1.5707963267948966) q[1];',
 'measure q[1] -> c[1];',
 'cx q[2],q[0];',
 'u2(0.0,3.141592653589793) q[2];',
 'u1(1.5707963267948966) q[2];',
 'measure q[2] -> c[2];',
 'measure q[0] -> c[0];',
 '']

In [47]:
len('bM|00>[1,2]')

11

In [88]:
a = circuit_names[22]
n = len(a)
print(a)
print(a[n-6:n])
eval(a[n-6:n])

bMZ2-Z1-HHS|00>[2, 1]
[2, 1]


[2, 1]

In [96]:
dict_cn = exptool.get_qasm_name_dict(compiled_qobj_list)

In [101]:
[c for c in exptool.CIRCUITS if '-'.join(reversed(c[0]))+c[1]==circuit_names[22][2:len(circuit_names[22])-6]][0]

[['HHS', 'Z1', 'Z2'], '|00>', [0.25, 0.25, 0.25, 0.25]]

In [74]:
'-'.join(reversed(exptool.CIRCUITS[13][0]))

'Z2'

In [75]:
circuit_names

['bMX2-CZ-HHS-X1|00>[1, 0]',
 'bMX2-CZ-HHS-X1|00>[2, 0]',
 'bMX2-CZ-HHS-X1|00>[2, 1]',
 'bMX2-CZ-HHS-X1|00>[2, 4]',
 'bMX2-CZ-HHS-X1|00>[3, 2]',
 'bMX2-CZ-HHS-X1|00>[3, 4]',
 'eMX2-CZ-HHS-X1|00>ftv1',
 'eMX2-CZ-HHS-X1|00>ftv2',
 'eMX2-CZ-HHS-X1|00>nftv1',
 'eMX2-CZ-HHS-X1|00>nftv2',
 'bMCZ-Z1-HHS|00>[1, 0]',
 'bMCZ-Z1-HHS|00>[2, 0]',
 'bMCZ-Z1-HHS|00>[2, 1]',
 'bMCZ-Z1-HHS|00>[2, 4]',
 'bMCZ-Z1-HHS|00>[3, 2]',
 'bMCZ-Z1-HHS|00>[3, 4]',
 'eMCZ-Z1-HHS|00>ftv1',
 'eMCZ-Z1-HHS|00>ftv2',
 'eMCZ-Z1-HHS|00>nftv1',
 'eMCZ-Z1-HHS|00>nftv2',
 'bMZ2-Z1-HHS|00>[1, 0]',
 'bMZ2-Z1-HHS|00>[2, 0]',
 'bMZ2-Z1-HHS|00>[2, 1]',
 'bMZ2-Z1-HHS|00>[2, 4]',
 'bMZ2-Z1-HHS|00>[3, 2]',
 'bMZ2-Z1-HHS|00>[3, 4]',
 'eMZ2-Z1-HHS|00>ftv1',
 'eMZ2-Z1-HHS|00>ftv2',
 'eMZ2-Z1-HHS|00>nftv1',
 'eMZ2-Z1-HHS|00>nftv2',
 'bMCZ-Z2-HHS|00>[1, 0]',
 'bMCZ-Z2-HHS|00>[2, 0]',
 'bMCZ-Z2-HHS|00>[2, 1]',
 'bMCZ-Z2-HHS|00>[2, 4]',
 'bMCZ-Z2-HHS|00>[3, 2]',
 'bMCZ-Z2-HHS|00>[3, 4]',
 'eMCZ-Z2-HHS|00>ftv1',
 'eMCZ-Z2-HHS|00>ftv2',
 'eM

In [82]:
l1 = [3, 4]
l2 = [6,7,8,9,10,11,12,13]
[l2[j] for j in l1]

[9, 10]

In [95]:
s = '01011'

In [167]:
importlib.reload(exptool)
dict_cn = exptool.get_qasm_name_dict(compiled_qobj_list)
j = 5
name = dict_cn['OPENQASM 2.0;'+job_result['qasms'][j]['qasm']][0]
n = len(name)
print(name[2:n-5])
d = exptool.api_data_to_dict(job_result['qasms'][j],name)
d

CZ-HHS|00>


{'counts': {'00': 1469,
  '01': 1124,
  '10': 1061,
  '11': 1478,
  'err': 3060,
  'total_valid': 5132},
 'expected_distribution_array': array([ 0.25,  0.25,  0.25,  0.25]),
 'experimental_distribution_array': array([ 0.28624318,  0.21901793,  0.20674201,  0.28799688]),
 'name': 'eMCZ-HHS|00>nftv1',
 'post_selection_ratio': 0.62646484375,
 'qasm_count': 17,
 'raw_counts': {'00000': 872,
  '00001': 14,
  '00010': 234,
  '00011': 7,
  '00100': 324,
  '00101': 8,
  '00110': 773,
  '00111': 7,
  '01000': 456,
  '01001': 10,
  '01010': 474,
  '01011': 1,
  '01100': 597,
  '01101': 7,
  '01110': 337,
  '01111': 2,
  '10000': 519,
  '10001': 5,
  '10010': 527,
  '10011': 2,
  '10100': 587,
  '10101': 5,
  '10110': 386,
  '10111': 5,
  '11000': 705,
  '11001': 7,
  '11010': 309,
  '11011': 2,
  '11100': 399,
  '11101': 6,
  '11110': 597,
  '11111': 8},
 'stand_dev': array([ 0.00630956,  0.00577321,  0.00565299,  0.00632108]),
 'stat_dist': 0.074240062353858147,
 'stat_dist_stand_dev': 0.008516

In [110]:
dict_cn

{'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncreg c[5];\nmeasure q[3] -> c[3];\nu2(0.0,3.141592653589793) q[2];\ncx q[2],q[4];\nmeasure q[2] -> c[2];\nmeasure q[4] -> c[4];\nmeasure q[1] -> c[1];\nmeasure q[0] -> c[0];\n': ['bM|00>+|11>[2, 4]'],
 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncreg c[5];\nmeasure q[3] -> c[3];\nu2(0.0,3.141592653589793) q[2];\ncx q[2],q[4];\nu1(3.141592653589793) q[4];\nmeasure q[4] -> c[4];\nmeasure q[2] -> c[2];\nmeasure q[1] -> c[1];\nmeasure q[0] -> c[0];\n': ['bMZ2|00>+|11>[2, 4]'],
 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncreg c[5];\nmeasure q[3] -> c[3];\nu2(0.0,3.141592653589793) q[2];\ncx q[2],q[4];\nu1(3.141592653589793) q[4];\nu3(3.141592653589793,0.0,3.141592653589793) q[4];\nmeasure q[4] -> c[4];\nmeasure q[2] -> c[2];\nmeasure q[1] -> c[1];\nmeasure q[0] -> c[0];\n': ['bMX2-Z2|00>+|11>[2, 4]'],
 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncreg c[5];\nmeasure q[3] -> c[3];\nu2(0.0,3.141592653589793) q[2];\ncx q[

In [127]:
l=[3,4]
l.reverse()

In [128]:
l

[4, 3]

In [153]:
'00000' in exptool.MAPPED_CODEWORDS[0]

True

In [165]:
"{0:02b}".format(3)

'11'

In [176]:
importlib.reload(exptool)
dict_cn = exptool.get_qasm_name_dict(compiled_qobj_list)
complete_dict = exptool.process_api_dump('data/API_dumps/api_dump_025788094baa89599b0ab43960bffe9d.txt',dict_cn)
complete_dict

{'bMCZ-HHS|00>[2, 0]': [{'calibration': {'fridgeParameters': {'Temperature': {'date': '2017-10-11T11:08:37Z',
      'unit': 'K',
      'value': 0.021},
     'cooldownDate': '2017-09-07'},
    'lastUpdateDate': '2017-10-11T11:08:37.000Z',
    'multiQubitGates': [{'gateError': {'date': '2017-10-11T11:08:37Z',
       'value': 0.02704530164528432},
      'name': 'CX1_0',
      'qubits': [1, 0],
      'type': 'CX'},
     {'gateError': {'date': '2017-10-11T11:08:37Z',
       'value': 0.026162842309928852},
      'name': 'CX2_0',
      'qubits': [2, 0],
      'type': 'CX'},
     {'gateError': {'date': '2017-10-11T11:08:37Z',
       'value': 0.02555222392427725},
      'name': 'CX2_1',
      'qubits': [2, 1],
      'type': 'CX'},
     {'gateError': {'date': '2017-10-11T11:08:37Z',
       'value': 0.03330587714106398},
      'name': 'CX2_4',
      'qubits': [2, 4],
      'type': 'CX'},
     {'gateError': {'date': '2017-10-11T11:08:37Z',
       'value': 0.022909177794157715},
      'name': 'CX3_

In [178]:
print(list(complete_dict.keys()))
len(complete_dict.keys())

['eMCZ-Z1-HHS|00>ftv2', 'eMZ2|00>+|11>', 'eMX2-Z2|00>+|11>', 'bM|00>[3, 4]', 'bM|00>[2, 1]', 'bM|00>[3, 2]', 'bM|00>[2, 0]', 'bM|00>[2, 4]', 'bM|00>[1, 0]', 'bM|0+>[1, 0]', 'bM|0+>[2, 0]', 'bMZ2|00>+|11>[3, 4]', 'bMZ2-HHS|00>[2, 1]', 'bMZ1-HHS|00>[2, 1]', 'bMZ2-Z1-HHS|00>[3, 4]', 'bMCZ-HHS|00>[3, 4]', 'bMZ1-HHS|00>[1, 0]', 'bMZ2-HHS|00>[1, 0]', 'eMZ1-HHS|00>ftv1', 'bMZ1-HHS|00>[3, 4]', 'bMZ2-HHS|00>[3, 4]', 'eMHHS|00>nftv2', 'bMZ2-X1|0+>[2, 0]', 'bM|00>+|11>[3, 4]', 'bMX2-CZ-HHS-X1|00>[3, 4]', 'eMZ2|0+>', 'bMX2-X1|00>[1, 0]', 'eMCZ-Z2-HHS|00>nftv2', 'bMZ2-X1|0+>[3, 2]', 'bMCZ-Z2-HHS|00>[2, 4]', 'bMCZ-Z1-HHS|00>[2, 4]', 'eMCZ-HHS|00>nftv1', 'bMZ2-Z1-HHS|00>[3, 2]', 'bMCZ-Z2-HHS|00>[3, 2]', 'bMCZ-Z1-HHS|00>[3, 2]', 'bMHHS|00>[1, 0]', 'bM|00>+|11>[3, 2]', 'bMX1|0+>[2, 1]', 'bM|0+>[2, 1]', 'eMX1|00>ftv1', 'bM|00>+|11>[2, 4]', 'bMX1|0+>[1, 0]', 'bMCZ-Z2-HHS|00>[3, 4]', 'bMCZ-Z1-HHS|00>[3, 4]', 'eMZ2-Z1-HHS|00>ftv2', 'bMX2|00>[2, 4]', 'bMX2|00>[3, 4]', 'eM|00>ftv1', 'eMCZ-Z1-HHS|00>nftv2', '

65